In [1]:
!pip install dash
!pip install dash-table

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 18.3 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3


In [58]:
!export GOOGLE_APPLICATION_CREDENTIALS="/content/drive/MyDrive/MLOps_testing/black-resource-440218-c3-5c0b7f31ce1f.json"
from sklearn.metrics import precision_score, recall_score, f1_score
from google.cloud import storage
from google.oauth2 import service_account
from PIL import Image
import os
import pandas as pd
import json
from datetime import timedelta

import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
from datetime import datetim
import pytz

from google.colab import drive
drive.mount('/content/drive')

credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/MLOps_testing/black-resource-440218-c3-5c0b7f31ce1f.json')
storage_client = storage.Client(credentials=credentials, project='black-resource-440218-c3')
bucket_name = 'nih-dataset-mlops'
bucket = storage_client.get_bucket(bucket_name)

# List all blobs (files) in the bucket
blobs = bucket.list_blobs()

# Print the file names (blobs) in the bucket
# for blob in blobs:
#     print(blob.name)
file_path = 'feedback/feedback.jsonl'

# Get the blob (file) from the bucket
blob = bucket.blob(file_path)

# Download the file content as a string
file_content = blob.download_as_text()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [59]:
# import json
# Read the content of the JSONL file
def process_jsonl_data(file_content):
    data = []
    for line in file_content.splitlines():
        data.append(json.loads(line))
    return data

# Process the data
data = process_jsonl_data(file_content)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)
print(df)

# Expand the predicted labels, true labels, and confidence scores into separate rows for easier analysis
expanded_data = []
for index, row in df.iterrows():
    for i, label in enumerate(row['predicted_labels']):

        # if not isinstance(row['true_labels'][i], list):
        #     true_label = [row['true_labels'][i]]
        # else:
        #     true_label = row['true_labels'][i]

        expanded_data.append({
            'image_name': row['image_name'],
            'age': row['age'],
            'gender': row['gender'],
            'predicted_label': label,
            # 'true_label': true_label,
            'confidence_score': row['confidence_score'][i],
            'feedback_type': row['feedback_type']
        })

expanded_df = pd.DataFrame(expanded_data)
print(expanded_df)

          image_name                                     image_metadata  age  \
0   00000001_000.png  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...   70   
1   00000001_001.png  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...   69   
2   00000001_002.png  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...   74   
3   00000003_000.png  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...   74   
4   00000003_001.png  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...   66   
..               ...                                                ...  ...   
56  00001088_025.png  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...   67   
57  00000740_000.png  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...   69   
58  00000632_010.png  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...   61   
59  00000019_000.png  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...   50   
60  00000798_040.png  /9j/4AAQSkZJRgABAQAAAQABAAD/2wBDAAgGBgcGBQgHBw...   50   

   gender                              

In [60]:
# Additional Metrics Calculation
# 1. Overall Inference Count and Count for Every Disease
overall_inference_count = expanded_df['predicted_label'].value_counts()
print(overall_inference_count)
print('Overall Inference: ',len(expanded_df))

# 2. Correct and Incorrect Inferences for Every Disease
# expanded_df['is_correct'] = expanded_df[expanded_df['predicted_label'] == expanded_df['true_label']]
# Group by predicted_label and feedback_type, and count the occurrences
correct_inference_count = expanded_df[expanded_df['feedback_type'] == 'Correct_Predictions'].groupby('predicted_label').size()
incorrect_inference_count = expanded_df[expanded_df['feedback_type'] == 'Incorrect_Predictions'].groupby('predicted_label').size()

print("Correct Inference Count:")
print(correct_inference_count)

print("\nIncorrect Inference Count:")
print(incorrect_inference_count)


# 3. Confidence Scores for Correct Inferences
correct_confidence_scores = expanded_df[expanded_df['feedback_type'] == 'Correct_Predictions'].groupby('predicted_label')['confidence_score'].mean()


predicted_label
Pleural_Thickening    51
No Finding            30
Cardiomegaly          16
Emphysema             15
Nodule                14
Infiltration           9
Hernia                 3
Mass                   1
Name: count, dtype: int64
Overall Inference:  139
Correct Inference Count:
predicted_label
Cardiomegaly          12
Emphysema             12
Hernia                 3
Infiltration           6
No Finding            19
Nodule                 8
Pleural_Thickening    33
dtype: int64

Incorrect Inference Count:
predicted_label
Cardiomegaly           4
Emphysema              3
Infiltration           3
Mass                   1
No Finding            11
Nodule                 6
Pleural_Thickening    18
dtype: int64


In [61]:
# 1. Create a function to calculate precision, recall, and F1 score
def calculate_metrics(df):
    metrics = {}

    # Get the unique predicted labels (diseases)
    predicted_labels = df['predicted_label'].unique()

    for label in predicted_labels:
        # Get True Positives (TP), False Positives (FP), False Negatives (FN)
        tp = len(df[(df['predicted_label'] == label) & (df['feedback_type'] == 'Correct_Predictions')])
        fp = len(df[(df['predicted_label'] == label) & (df['feedback_type'] == 'Incorrect_Predictions')])
        fn = len(df[(df['predicted_label'] != label) & (df['feedback_type'] == 'Incorrect_Predictions')])

        # Compute precision, recall, and F1 score for each label
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

        overall_inference_count = len(df[df['predicted_label'] == label])
        # print(overall_inference_count)
        # print('Overall Inference: ',len(expanded_df))


        correct_inference_count = len(df[(df['feedback_type'] == 'Correct_Predictions') & (df['predicted_label'] == label)])
        incorrect_inference_count = len(df[(df['feedback_type'] == 'Incorrect_Predictions') & (df['predicted_label'] == label)])
        avg_correct_inference_confidence = df[(df['feedback_type'] == 'Correct_Predictions') & (df['predicted_label'] == label)].confidence_score.mean()
        # avg_incorrect_inference_count = df[(df['feedback_type'] == 'Incorrect_Predictions') & (df['predicted_label'] == label)].confidence_score.mean()

        # print("Correct Inference Count:")
        # print(correct_inference_count)

        # print("\nIncorrect Inference Count:")
        # print(incorrect_inference_count)

        # Store the metrics
        metrics[label] = {
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'overall_inference_count': overall_inference_count,
            'correct_inference_count': correct_inference_count,
            'incorrect_inference_count': incorrect_inference_count,
            'avg_correct_inference_confidence': avg_correct_inference_confidence,
            # 'avg_incorrect_inference_count': avg_incorrect_inference_count
        }


    metrics['total'] = {
        'precision': None,
        'recall': None,
        'f1_score': None,
        'overall_inference_count': len(df),
        'correct_inference_count': len(df[df['feedback_type'] == 'Correct_Predictions']),
        'incorrect_inference_count': len(df[df['feedback_type'] == 'Incorrect_Predictions']),
        'avg_correct_inference_confidence': None,
        # 'avg_incorrect_inference_count': avg_incorrect_inference_count
    }

    return metrics

# Calculate metrics
metrics = calculate_metrics(expanded_df)

# Convert metrics to DataFrame for easier visualization
metrics_df = pd.DataFrame.from_dict(metrics, orient='index')

# Display the metrics for each disease
print(metrics_df)

                    precision    recall  f1_score  overall_inference_count  \
Pleural_Thickening   0.647059  0.540984  0.589286                       51   
No Finding           0.633333  0.351852  0.452381                       30   
Cardiomegaly         0.750000  0.222222  0.342857                       16   
Emphysema            0.800000  0.218182  0.342857                       15   
Nodule               0.571429  0.166667  0.258065                       14   
Hernia               1.000000  0.061224  0.115385                        3   
Mass                 0.000000  0.000000  0.000000                        1   
Infiltration         0.666667  0.122449  0.206897                        9   
total                     NaN       NaN       NaN                      139   

                    correct_inference_count  incorrect_inference_count  \
Pleural_Thickening                       33                         18   
No Finding                               19                         11 

In [107]:
from datetime import datetime
import pytz
est_timezone = pytz.timezone('US/Eastern')
timestamp = datetime.now(est_timezone).strftime("%Y-%m-%d_%H-%M-%S")
# print(timestamp)
metrics_df.to_csv(f'/content/drive/MyDrive/MLOps_testing/monitoring/metrics_{timestamp}.csv')

bucket_name = 'nih-dataset-mlops'
destination_blob_name = f'model_monitoring/metrics_{timestamp}.csv'


csv_buffer = io.StringIO()
metrics_df.to_csv(csv_buffer, index=False)
# Rewind the buffer's cursor to the start
csv_buffer.seek(0)

# Create a blob (file object) and upload the in-memory CSV to GCS
blob = bucket.blob(destination_blob_name)
blob.upload_from_file(csv_buffer, content_type='text/csv')

print(f"File uploaded to gs://{bucket_name}/{destination_blob_name}")

File uploaded to gs://nih-dataset-mlops/model_monitoring/metrics_2024-12-08_23-35-58.csv


In [63]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output


# Create Dash app
app = dash.Dash(__name__)

# Layout with Tabs
app.layout = html.Div([
    html.H1("Model Performance Dashboards"),

    # Tabs
    dcc.Tabs([
        # First Tab: Overall Inference and Correct/Incorrect Inferences
        dcc.Tab(label='Overall Inference Metrics', children=[
            html.H2("Overall Model Inference Metrics"),
            dcc.Graph(
                id='disease-inference-count',
                figure=px.bar(overall_inference_count, title="Overall Inference Count", labels={'index': 'Disease', 'value': 'Inference Count'})
            ),
            dcc.Graph(
                id='correct-incorrect-inferences',
                figure=px.bar(
                    pd.DataFrame({
                        'correct': correct_inference_count,
                        'incorrect': incorrect_inference_count
                    }).reset_index().melt(id_vars='predicted_label'),
                    x='predicted_label', y='value', color='variable', title="Correct vs Incorrect Inferences (Based on Feedback)"
                )
            ),
            dcc.Graph(
                id='confidence-scores',
                figure=px.box(
                    expanded_df[expanded_df['feedback_type'] == 'Correct_Predictions'], x='predicted_label', y='confidence_score',
                    title="Confidence Scores for Correct Inferences"
                )
            )
        ]),

        # Second Tab: Disease-Specific Metrics
        dcc.Tab(label='Disease Performance', children=[
            html.H2("Select Disease to View Metrics"),
            dcc.Dropdown(
                id='disease-dropdown',
                options=[{'label': disease, 'value': disease} for disease in metrics_df.index[:-1]],
                value='Pleural_Thickening',  # Default value
                style={'width': '50%'}
            ),
            html.Div(id='disease-dashboard')
        ])
    ])
])

# Callback to update the disease dashboard based on selected disease
@app.callback(
    Output('disease-dashboard', 'children'),
    [Input('disease-dropdown', 'value')]
)
def update_dashboard(disease):
    disease_data = metrics_df.loc[disease]

    # Create the disease-specific dashboard
    return html.Div([
        html.H2(f"Metrics for {disease}"),
        dash_table.DataTable(
            columns=[
                {"name": "Metric", "id": "metric"},
                {"name": "Value", "id": "value"}
            ],
            data=[
                {"metric": "Precision", "value": disease_data['precision']},
                {"metric": "Recall", "value": disease_data['recall']},
                {"metric": "F1 Score", "value": disease_data['f1_score']},
                {"metric": "Overall Inference Count", "value": disease_data['overall_inference_count']},
                {"metric": "Correct Inference Count", "value": disease_data['correct_inference_count']},
                {"metric": "Incorrect Inference Count", "value": disease_data['incorrect_inference_count']}
            ],
            style_table={'height': '300px', 'overflowY': 'auto'}
        ),
        # Bar chart for Correct vs Incorrect Inferences
        dcc.Graph(
            id=f'correct-incorrect-{disease}',
            figure={
                'data': [
                    go.Bar(
                        x=['Correct', 'Incorrect'],
                        y=[disease_data['correct_inference_count'], disease_data['incorrect_inference_count']],
                        name=f'Inferences for {disease}',
                        marker=dict(color=['green', 'red'])
                    )
                ],
                'layout': go.Layout(
                    title=f'Correct and Incorrect Inferences for {disease}',
                    xaxis={'title': 'Inference Type'},
                    yaxis={'title': 'Count'}
                )
            }
        )
    ])

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

In [86]:
import glob
import io

# Function to extract timestamp from the filename
def extract_timestamp_from_filename(filename):
    # Assuming timestamp is part of the filename in the format "filename_YYYY-MM-DD_HH-MM-SS.csv"
    # print(filename)
    base_name = os.path.basename(filename)
    timestamp_str = base_name.split('_')[1] + '_' + base_name.split('_')[-1].replace('.csv', '')  # Assuming the timestamp is the last part before ".csv"

    print(filename)
    print(base_name)
    print(timestamp_str)
    print(datetime.strptime(timestamp_str, "%Y-%m-%d_%H-%M-%S"))
    return datetime.strptime(timestamp_str, "%Y-%m-%d_%H-%M-%S")

# Path to the directory containing your CSV files
directory_path = '/content/drive/MyDrive/MLOps_testing/monitoring'

# Get a list of all CSV files in the directory
csv_files = glob.glob(os.path.join(directory_path, "*.csv"))
print(csv_files)

['/content/drive/MyDrive/MLOps_testing/monitoring/metrics_2024-12-08_20-37-44.csv', '/content/drive/MyDrive/MLOps_testing/monitoring/metrics_2024-12-08_20-41-51.csv', '/content/drive/MyDrive/MLOps_testing/monitoring/metrics_2024-12-08_21-48-39.csv']


In [103]:
credentials = service_account.Credentials.from_service_account_file('/content/drive/MyDrive/MLOps_testing/black-resource-440218-c3-5c0b7f31ce1f.json')
storage_client = storage.Client(credentials=credentials, project='black-resource-440218-c3')
bucket_name = 'nih-dataset-mlops'
bucket = storage_client.get_bucket(bucket_name)

# List all blobs (files) in the bucket
path_prefix = 'model_monitoring'
blobs = bucket.list_blobs(prefix=path_prefix)

# Initialize an empty list to store DataFrames
dfs = []
df = pd.DataFrame()

# Print the file names (blobs) in the bucket
for blob in blobs:
    if blob.name.endswith('.csv'):
      print(blob.name)  # Check if it's a CSV file
      # Download the content of the file as text
      file_content = blob.download_as_text()

      # Use StringIO to read the content into a pandas DataFrame
      data = pd.read_csv(io.StringIO(file_content))

      # Optionally, add a column for the filename
      data['filename'] = blob.name
      timestamp = extract_timestamp_from_filename(blob.name)
      data['timestamp'] = timestamp
      df = pd.concat([df,data], ignore_index=True)
      dfs.append(df)

      # Now you can work with the DataFrame `data` directly, for example:
      print(f"Data from {blob.name}:")
      print(data.head())
# print(dfs)
# # Concatenate all DataFrames into a single DataFrame
# final_df = pd.concat(dfs, ignore_index=True)

# # Optionally, you can save the combined DataFrame to a new CSV file
# # final_df.to_csv('/path/to/save/combined_data.csv', index=False)

# # Display the first few rows of the combined DataFrame
# final_df.columns = ['predicted_disease', 'precision', 'recall', 'f1_score',
#        'overall_inference_count', 'correct_inference_count',
#        'incorrect_inference_count','avg_correct_inference_confidence', 'filename', 'timestamp']
# # print(final_df.head())
# print(final_df)


final_df = df.copy(deep=True)

final_df = final_df.rename(columns={'Unnamed: 0': 'predicted_disease'})
print(final_df)

model_monitoring/metrics_2024-12-08_20-37-44.csv
model_monitoring/metrics_2024-12-08_20-37-44.csv
metrics_2024-12-08_20-37-44.csv
2024-12-08_20-37-44
2024-12-08 20:37:44
Data from model_monitoring/metrics_2024-12-08_20-37-44.csv:
           Unnamed: 0  precision    recall  f1_score  overall_inference_count  \
0  Pleural_Thickening   0.632653  0.525424  0.574074                       49   
1          No Finding   0.633333  0.351852  0.452381                       30   
2        Cardiomegaly   0.733333  0.207547  0.323529                       15   
3           Emphysema   0.785714  0.203704  0.323529                       14   
4              Nodule   0.571429  0.166667  0.258065                       14   

   correct_inference_count  incorrect_inference_count  \
0                       31                         18   
1                       19                         11   
2                       11                          4   
3                       11                          3  

In [65]:
# # Initialize an empty list to store DataFrames
# dfs = []

# # Loop through each CSV file
# for file in csv_files:
#     # Read the CSV file into a DataFrame
#     df = pd.read_csv(file)

#     # Extract the timestamp from the filename
#     timestamp = extract_timestamp_from_filename(file)

#     # Add new columns to store the filename and timestamp
#     df['filename'] = os.path.basename(file)  # Add filename column
#     df['timestamp'] = timestamp  # Add timestamp column

#     # Append the current DataFrame to the list
#     dfs.append(df)

# # Concatenate all DataFrames into a single DataFrame
# final_df = pd.concat(dfs, ignore_index=True)

# # Optionally, you can save the combined DataFrame to a new CSV file
# # final_df.to_csv('/path/to/save/combined_data.csv', index=False)

# # Display the first few rows of the combined DataFrame
# final_df.columns = ['predicted_disease', 'precision', 'recall', 'f1_score',
#        'overall_inference_count', 'correct_inference_count',
#        'incorrect_inference_count', 'filename', 'timestamp',
#        'avg_correct_inference_confidence']
# print(final_df.head())
# print(final_df)

    predicted_disease  precision    recall  f1_score  overall_inference_count  \
0  Pleural_Thickening   0.632653  0.525424  0.574074                       49   
1          No Finding   0.633333  0.351852  0.452381                       30   
2        Cardiomegaly   0.733333  0.207547  0.323529                       15   
3           Emphysema   0.785714  0.203704  0.323529                       14   
4              Nodule   0.571429  0.166667  0.258065                       14   

   correct_inference_count  incorrect_inference_count  \
0                       31                         18   
1                       19                         11   
2                       11                          4   
3                       11                          3   
4                        8                          6   

                          filename           timestamp  \
0  metrics_2024-12-08_20-37-44.csv 2024-12-08 20:37:44   
1  metrics_2024-12-08_20-37-44.csv 2024-12-08 20:37:44

In [104]:
from datetime import timedelta

# Assuming 'final_df' is the DataFrame containing your combined data

# Ensure the timestamp column is in datetime format
final_df1 = final_df.copy(deep=True)
final_df1['timestamp'] = pd.to_datetime(final_df1['timestamp'], format='%Y-%m-%d %H:%M:%S')

# Sort by timestamp to ensure the data is in chronological order
final_df1 = final_df1.sort_values(by='timestamp')

# Get the unique diseases
diseases = final_df1['predicted_disease'].unique()
print(final_df1)

     predicted_disease  precision    recall  f1_score  \
0   Pleural_Thickening   0.632653  0.525424  0.574074   
1           No Finding   0.633333  0.351852  0.452381   
2         Cardiomegaly   0.733333  0.207547  0.323529   
3            Emphysema   0.785714  0.203704  0.323529   
4               Nodule   0.571429  0.166667  0.258065   
5               Hernia   1.000000  0.061224  0.115385   
6                 Mass   0.000000  0.000000  0.000000   
7         Infiltration   0.625000  0.104167  0.178571   
8                total        NaN       NaN       NaN   
17               total        NaN       NaN       NaN   
16        Infiltration   0.666667  0.122449  0.206897   
15                Mass   0.000000  0.000000  0.000000   
14              Hernia   1.000000  0.061224  0.115385   
13              Nodule   0.571429  0.166667  0.258065   
12           Emphysema   0.800000  0.218182  0.342857   
11        Cardiomegaly   0.750000  0.222222  0.342857   
10          No Finding   0.6333

In [98]:
# Loop through each disease and plot the graph
for disease in diseases:
    # Filter the data for the current disease
    disease_data = final_df1[final_df1['predicted_disease'] == disease]
    print(disease_data)

     predicted_disease  precision    recall  f1_score  \
0   Pleural_Thickening   0.647059  0.540984  0.589286   
18  Pleural_Thickening   0.647059  0.540984  0.589286   
9   Pleural_Thickening   0.647059  0.540984  0.589286   

    overall_inference_count  correct_inference_count  \
0                        51                       33   
18                       51                       33   
9                        51                       33   

    incorrect_inference_count  avg_correct_inference_confidence  \
0                          18                          0.537538   
18                         18                          0.537538   
9                          18                          0.537538   

                           filename           timestamp  
0   metrics_2024-12-08_21-48-39.csv 2024-12-08 21:48:39  
18  metrics_2024-12-08_21-48-39.csv 2024-12-08 21:48:39  
9   metrics_2024-12-08_21-48-39.csv 2024-12-08 21:48:39  
   predicted_disease  precision  recall  f1_s

In [105]:
# import pandas as pd
# import plotly.express as px
from datetime import timedelta

# Assuming 'final_df' is the DataFrame containing your combined data

# Ensure the timestamp column is in datetime format
final_df1 = final_df.copy(deep=True)
final_df1['timestamp'] = pd.to_datetime(final_df1['timestamp'], format='%Y-%m-%d %H:%M:%S')

# Sort by timestamp to ensure the data is in chronological order
final_df1 = final_df1.sort_values(by='timestamp')

# Get the unique diseases
diseases = final_df1['predicted_disease'].unique()

# Loop through each disease and plot the graph
for disease in diseases:
    # Filter the data for the current disease
    disease_data = final_df1[final_df1['predicted_disease'] == disease]

    # Plotting recall and f1_score over time for the current disease
    fig = px.line(disease_data,
                  x='timestamp',
                  y=['recall', 'f1_score'],
                  title=f"{disease} - Recall and F1 Score Improvement Over Time",
                  labels={'timestamp': 'Time', 'value': 'Score'},
                  line_shape='linear')

    min_timestamp = disease_data['timestamp'].min()
    max_timestamp = disease_data['timestamp'].max()

    extended_min_timestamp = min_timestamp - timedelta(hours=0.5)  # 30 days in the past
    extended_max_timestamp = max_timestamp + timedelta(hours=0.5)

    # Update x-axis to show bigger scale by spacing out the ticks
    fig.update_xaxes(
        range=[extended_min_timestamp, extended_max_timestamp],
        tickmode='auto',  # Auto mode will dynamically determine the best ticks
        tickformat='%Y-%m-%d',  # Format the timestamp as 'YYYY-MM-DD'
        dtick="W1",  # Use weekly ticks for a bigger scale
        tickangle=45,  # Rotate the ticks for better readability
    )

    # Show the plot
    fig.show()


In [90]:
final_df1['timestamp']

,timestamp
0,metrics_2024-12-08_21-48-39.csv
1,metrics_2024-12-08_21-48-39.csv
2,metrics_2024-12-08_21-48-39.csv
3,metrics_2024-12-08_21-48-39.csv
4,metrics_2024-12-08_21-48-39.csv
5,metrics_2024-12-08_21-48-39.csv
6,metrics_2024-12-08_21-48-39.csv
7,metrics_2024-12-08_21-48-39.csv
8,metrics_2024-12-08_21-48-39.csv
9,metrics_2024-12-08_21-48-39.csv


In [106]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
from dash.dependencies import Input, Output
from datetime import timedelta

# Assuming 'final_df' is the DataFrame containing your combined data

# Create a Dash app
app = dash.Dash(__name__)

# Ensure the timestamp column is in datetime format
final_df1 = final_df.copy(deep=True)
final_df1['timestamp'] = pd.to_datetime(final_df1['timestamp'])

# Sort by timestamp to ensure the data is in chronological order
final_df1 = final_df1.sort_values(by='timestamp')

# Get the unique diseases
diseases = final_df1['predicted_disease'].unique()

# Layout for the dashboard
app.layout = html.Div([
    html.H1("Disease Recall and F1 Score Improvement Over Time"),

    # Dropdown to select a disease
    dcc.Dropdown(
        id='disease-dropdown',
        options=[{'label': disease, 'value': disease} for disease in diseases],
        value=diseases[0],  # Default value (first disease)
        style={'width': '50%'}
    ),

    # Graph to show recall and F1 score
    dcc.Graph(id='disease-graph')
])

# Callback to update the graph based on selected disease
@app.callback(
    Output('disease-graph', 'figure'),
    [Input('disease-dropdown', 'value')]
)
def update_graph(selected_disease):
    # Filter the data for the selected disease
    disease_data = final_df1[final_df1['predicted_disease'] == selected_disease]

    # Plotting recall and f1_score over time for the selected disease
    fig = px.line(disease_data,
                  x='timestamp',
                  y=['recall', 'f1_score'],
                  title=f"{selected_disease} - Recall and F1 Score Improvement Over Time",
                  labels={'timestamp': 'Time', 'value': 'Score'},
                  line_shape='linear')

    # Get the min and max timestamp for the selected disease
    min_timestamp = disease_data['timestamp'].min()
    max_timestamp = disease_data['timestamp'].max()

    # Extend the timeline by 0.5 hours before and after the data range
    extended_min_timestamp = min_timestamp - timedelta(hours=0.5)
    extended_max_timestamp = max_timestamp + timedelta(hours=0.5)

    # Update x-axis to show a bigger scale with weekly ticks
    fig.update_xaxes(
        range=[extended_min_timestamp, extended_max_timestamp],
        tickmode='auto',  # Auto mode will dynamically determine the best ticks
        tickformat='%Y-%m-%d',  # Format the timestamp as 'YYYY-MM-DD'
        dtick="W1",  # Use weekly ticks for a bigger scale
        tickangle=45,  # Rotate the ticks for better readability
    )

    # Return the figure to be displayed
    return fig

# Run the Dash app
if __name__ == '__main__':
    app.run_server(debug=True)


<IPython.core.display.Javascript object>

In [ ]:
# 3. Confidence Scores for Correct Inferences
correct_confidence_scores = expanded_df[expanded_df['feedback_type'] == 'Correct_Predictions'].groupby('predicted_label')['confidence_score'].mean()

In [11]:
# import plotly.express as px
# fig = px.box(data, x='predicted_label', y='confidence_score', title="Confidence Scores for Correct Predictions")

figure=px.box(
    expanded_df[expanded_df['feedback_type'] == 'Correct_Predictions'], x='predicted_label', y='confidence_score',
    title="Confidence Scores for Correct Inferences"
)
figure.show()

In [13]:
fig = px.histogram(expanded_df[expanded_df['feedback_type'] == 'Correct_Predictions'], x='predicted_label', nbins=20, title="Distribution of Confidence Scores")
fig.show()


In [17]:
# fig = px.density_contour(expanded_df[expanded_df['feedback_type'] == 'Correct_Predictions'], x='predicted_label', title="Confidence Score Density")
data = expanded_df[expanded_df['feedback_type'] == 'Correct_Predictions']
fig = px.violin(data, x='predicted_label', y='confidence_score', box=True, points="all", title="Violin Plot of Confidence Scores")

fig.show()


In [20]:
fig = px.scatter(data, x='predicted_label', y='confidence_score', color='feedback_type', title="Confidence Score vs Predicted Label")
fig.show()